In [1]:
from packages.transcripts import transcribe_with_timestamps
from packages.PII_detection import detect_pii_entities
from packages.mapTimeStamps import map_pii_to_timestamps
from packages.saveRedactedAudo import redact_audio

In [2]:
input_audio_path = "inputs/in-70001-919417328881-20250530-160219-1748601139.42162.wav"

### 🎙️ Step 1: Transcribe Hindi Audio with Word-Level Timestamps

This function uses Azure's Speech SDK to transcribe a WAV audio file in Hindi (`hi-IN`) and extract:

- Recognized speech as text.
- Word-level timestamps including start and end time (in seconds).

The results are collected and returned in a structured dictionary with `text` and `chunks`.


In [ ]:
transcript = transcribe_with_timestamps(input_audio_path)
print(transcript)

{'text': 'हैलो हाँ जी हाँ जी सर फ़ोन आया था, फ़ोन आया था, एक सेकंड होल्ड करो जी मैं नाम जान सकती हूँ थोड़ा राजेंद्र सिंह जी। राजेंद्र सिंह, राजेंद्र सिंह, राजेंद्र। एलए जे आई एनडी वि आर ओके ओके ओके ओके राजेंद्र सिंह जी, मैं कल्टीमेट कंपनी पटियाले तो कल कर रही हूँ। थोड़े कॉल है ना छोटा कोइना को फील्ड ऑफिसर आया होना उन्हें थोड़ी ना कार्बन क्रेडिट प्रोग्राम बारे जानकारी देती होनी सर। जुड़े खेतों के बीच पाइप लग गए न सर। पाइप का मैसेज लाउड देने। नहीं, मेरे को कोई नहीं आएगी अजय जी। मुझे नहीं आएगी, थोड़े अच्छा जी है क्या वो मशीन है वो वो करी जानते है ना जी साधु मशीन नाल तो। जसवंत सिंह जी ने जो दुस्य होना ये तो प्रॉपर 10 नहीं पाए या तो उसे समझ नहीं पाए मैं दुबारा 10 जन नहीं आ, थोड़ी खेत दे विच ना सर। एक पाइप में मैं सेंसर लाया जाता है। देसी भाषा जो ना पाइप गया था नहीं ऐसी कंपनी लेवल का ना सेंसर कह देना। ठीक है जी, पार्टनर लग दूंगा। पहली का तो थोड़ी फसिलिटी कोई नुक्सान नहीं करूँगा। कोई जगह नहीं ज्यादा खेलू कहरूंगा ठीक है जी करूँगा की जो दो थोड़े खेत के बीच पानी भी मात्रा कट चुकी अपने फ़ोन पे मै

### 🕵️ Step 2: Detect PII Entities in Hindi Text

This function uses the Azure client to recognize PII entities in a Hindi document. It:

- Calls Azure's `recognize_pii_entities()` method with language set to `"hi"` (Hindi).
- Prints redacted text and detailed info for each detected entity.
- Returns a list of all PII words detected in the input.


In [ ]:
asr_text = [transcript['text']]
pii_result = detect_pii_entities(asr_text)
pii_words = []
for doc in pii_result:
    print("Redacted Text: {}".format(doc.redacted_text))
    for entity in doc.entities:
        print("Entity: {}".format(entity.text))
        pii_words.append(entity.text)
        print("\tCategory: {}".format(entity.category))
        print("\tConfidence Score: {}".format(entity.confidence_score))
        print("\tOffset: {}".format(entity.offset))
        print("\tLength: {}".format(entity.length))

Redacted Text: हैलो हाँ जी हाँ जी सर फ़ोन आया था, फ़ोन आया था, एक सेकंड होल्ड करो जी मैं नाम जान सकती हूँ थोड़ा ************* जी। *************, *************, ********। एलए जे आई एनडी वि आर ओके ओके ओके ओके ************* जी, मैं कल्टी********* पटियाले तो कल कर रही हूँ। थोड़े कॉल है ना छोटा ***** को फी********* आया होना उन्हें थोड़ी ना कार्बन क्रेडिट प्रोग्राम बारे जानकारी देती होनी सर। जुड़े खेतों के बीच पाइप लग गए न सर। पाइप का मैसेज लाउड देने। नहीं, मेरे को कोई नहीं आएगी *** जी। मुझे नहीं आएगी, थोड़े अच्छा जी है क्या वो मशीन है वो वो करी जानते है ना जी साधु मशीन नाल तो। ********** जी ने जो दुस्य होना ये तो प्रॉपर 10 नहीं पाए या तो उसे समझ नहीं पाए मैं दुबारा 10 जन नहीं आ, थोड़ी खेत दे विच ना सर। एक पाइप में मैं सेंसर लाया जाता है। देसी भाषा जो ना पाइप गया था नहीं ऐसी कंपनी लेवल का ना सेंसर कह देना। ठीक है जी, ******* लग दूंगा। पहली का तो थोड़ी फसिलिटी कोई नुक्सान नहीं करूँगा। कोई जगह नहीं ज्यादा खेलू कहरूंगा ठीक है जी करूँगा की जो दो थोड़े खेत के बीच पानी भी मात्रा कट चुकी अपने फ़ोन 

### 🧠 Step 3: Map PII Words to Audio Word Timestamps

This function attempts to align each detected PII word to a timestamped word chunk in the ASR output using:

- Fuzzy string matching (`SequenceMatcher`) to allow for slight mismatches.
- Returns a list of timestamps corresponding to each PII word.

This is crucial for knowing **where** in the audio to apply redaction.


In [ ]:
pii_segments = map_pii_to_timestamps(pii_words, transcript['chunks'], threshold=0.6)
print(pii_segments)

[(13.47, 13.91), (17.2, 17.72), (18.68, 19.32), (20.28, 20.68), (25.07, 25.55), (26.55, 26.91), (30.6, 30.92), (31.24, 31.68), (44.59, 44.79), (58.36, 59.32), (73.75, 74.43), (87.19, 87.71), (110.35, 110.87), (122.83, 123.15), (123.27, 123.79), (158.91, 159.19), (159.19, 159.47), (242.23, 242.35), (246.15, 247.23), (257.0, 257.28)]


### 🔇 Step 4: Mute Audio Segments Containing PII

This function takes the original audio file and the timestamps of detected PII, and:

- Loads the audio using `torchaudio`.
- Silences the waveform between each PII segment (sets amplitude to zero).
- Saves the redacted audio file to `output_audio/` directory, replacing `"in-"` with `"out-"` in the filename.


In [ ]:
redact_audio(input_audio_path,pii_segments)

✅ Redacted audio saved at: output_audio\out-70001-919417328881-20250530-160219-1748601139.42162.wav
